In [2]:
# try everything here
import sys
import networkx as nx # type: ignore
import graphviz
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from dowhy import CausalModel
import statsmodels.formula.api as smf

sys.path.append('../')
from src.data.make_dataset import load_data, merge_all_datasets
from src.data.preprocess import preprocessing

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [3]:
# Loading Data
data_dict = load_data()

olist_customers_df = data_dict['olist_customers_df']
olist_geolocation_df = data_dict['olist_geolocation_df']
olist_order_items_df = data_dict['olist_order_items_df']
olist_order_payments_df = data_dict['olist_order_payments_df']
olist_order_reviews_df = data_dict['olist_order_reviews_df']
olist_orders_df = data_dict['olist_orders_df']
olist_products_df = data_dict['olist_products_df']
olist_sellers_df = data_dict['olist_sellers_df']
product_category_name_translation_df = data_dict['product_category_name_translation_df']

olist_closed_deals_df = data_dict['olist_closed_deals_df']
olist_marketing_qualified_leads_df = data_dict['olist_marketing_qualified_leads_df']

In [4]:
df = merge_all_datasets(olist_customers_df, 
                       olist_geolocation_df,
                       olist_order_items_df,
                       olist_order_payments_df,
                       olist_order_reviews_df, 
                       olist_orders_df,
                       olist_products_df, 
                       olist_sellers_df
                       )

In [5]:
rainfall_categories = {
    'North': {1: 'High', 2: 'High', 3: 'High', 4: 'Moderate', 5: 'Moderate', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Moderate', 10: 'Moderate', 11: 'High', 12: 'High'},
    'Northeast': {1: 'Low', 2: 'Moderate', 3: 'Moderate', 4: 'High', 5: 'High', 6: 'Moderate', 7: 'Low', 8: 'Low', 9: 'Low', 10: 'Moderate', 11: 'Moderate', 12: 'High'},
    'Central-West': {1: 'High', 2: 'High', 3: 'High', 4: 'Moderate', 5: 'Low', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Low', 10: 'Moderate', 11: 'High', 12: 'High'},
    'Southeast': {1: 'High', 2: 'High', 3: 'Moderate', 4: 'Moderate', 5: 'Low', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Moderate', 10: 'Moderate', 11: 'High', 12: 'High'},
    'South': {1: 'Moderate', 2: 'Moderate', 3: 'Moderate', 4: 'Moderate', 5: 'Moderate', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Moderate', 10: 'Moderate', 11: 'Moderate', 12: 'High'}
}

state_to_region = {
    'AC': 'North', 'AP': 'North', 'AM': 'North', 'PA': 'North', 'RO': 'North', 'RR': 'North', 'TO': 'North',
    'AL': 'Northeast', 'BA': 'Northeast', 'CE': 'Northeast', 'MA': 'Northeast', 'PB': 'Northeast', 'PE': 'Northeast', 'PI': 'Northeast', 'RN': 'Northeast', 'SE': 'Northeast',
    'GO': 'Central-West', 'MT': 'Central-West', 'MS': 'Central-West', 'DF': 'Central-West',
    'ES': 'Southeast', 'MG': 'Southeast', 'RJ': 'Southeast', 'SP': 'Southeast',
    'PR': 'South', 'RS': 'South', 'SC': 'South'
}


def get_rainfall_category(region, month):

    rainfall_categories = {
    'North': {1: 'High', 2: 'High', 3: 'High', 4: 'Moderate', 5: 'Moderate', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Moderate', 10: 'Moderate', 11: 'High', 12: 'High'},
    'Northeast': {1: 'Low', 2: 'Moderate', 3: 'Moderate', 4: 'High', 5: 'High', 6: 'Moderate', 7: 'Low', 8: 'Low', 9: 'Low', 10: 'Moderate', 11: 'Moderate', 12: 'High'},
    'Central-West': {1: 'High', 2: 'High', 3: 'High', 4: 'Moderate', 5: 'Low', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Low', 10: 'Moderate', 11: 'High', 12: 'High'},
    'Southeast': {1: 'High', 2: 'High', 3: 'Moderate', 4: 'Moderate', 5: 'Low', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Moderate', 10: 'Moderate', 11: 'High', 12: 'High'},
    'South': {1: 'Moderate', 2: 'Moderate', 3: 'Moderate', 4: 'Moderate', 5: 'Moderate', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Moderate', 10: 'Moderate', 11: 'Moderate', 12: 'High'}
    }

    return rainfall_categories.get(region).get(month, 'Unknown')


In [6]:
final_df = preprocessing(df=df, state_to_region=state_to_region)

In [7]:
final_df.rename(columns={
    'seasonality': 'season',
    'review_score': 'Rating',
    'rainfall': 'region'
    
}, inplace=True)

/var/folders/7s/18428v6x0c9g1ltv9g71gm1h0000gn/T/ipykernel_78799/1841060352.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns={


In [8]:
df_final = final_df[['order_id', 
               'payment_value', 
               'Rating',
               'region', 
               'Product_weight_kg', 
               'distance_km',
               'Product_category', 
               'Product_size', 
               'No_photos',
               'Product_price', 
               'season', 
               'is_delivery_late',
               'Product_price',
               'freight_value',
               'Product_category_encoded']]

In [9]:
df_final

,order_id,payment_value,Rating,region,Product_weight_kg,distance_km,Product_category,Product_size,No_photos,Product_price,season,is_delivery_late,Product_price,freight_value,Product_category_encoded
0,d455a8cb295653b55abda06d434ab492,916.02,5.0,South,11.8,687.434185,eletroportateis,61920.0,2.0,895.0,9,0,895.0,21.02,31
1,7f39ba4c9052be115350065d07583cac,916.02,1.0,Southeast,11.8,417.139650,eletroportateis,61920.0,2.0,895.0,10,0,895.0,21.02,31
2,9dc8d1a6f16f1b89874c29c9d8d30447,916.02,5.0,Southeast,11.8,714.295670,eletroportateis,61920.0,2.0,895.0,10,0,895.0,21.02,31
3,0f548bd29148ebc519d6495e37a284e1,264.16,5.0,Southeast,9.0,389.663761,ferramentas_jardim,19656.0,1.0,99.0,9,0,99.0,33.08,40
4,0f548bd29148ebc519d6495e37a284e1,264.16,5.0,Southeast,9.0,389.663761,ferramentas_jardim,19656.0,1.0,99.0,9,0,99.0,33.08,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119138,87ae60ef8b08ae0e5f903cacb53a6904,258.67,1.0,Southeast,NaN,NaN,NaN,NaN,NaN,NaN,9,0,NaN,NaN,73
119139,d69fcef5a5fe3a3db60ea65c6ee499cc,69.38,4.0,Northeast,NaN,NaN,NaN,NaN,NaN,NaN,12,0,NaN,NaN,73
119140,f86d7bc39aab05299691322044b63bb2,1302.42,2.0,North,NaN,NaN,NaN,NaN,NaN,NaN,8,0,NaN,NaN,73
119141,74f833bf7c70ce8c3f820f725c213e1c,101.56,1.0,Northeast,NaN,NaN,NaN,NaN,NaN,NaN,6,0,NaN,NaN,73


In [10]:
df_final.to_csv("../data/processed/data.csv", index=False)